In [7]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.keras import backend as K
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
# Set up the tensorflow session as same as the keras session
K.set_session(sess)

In [8]:
# Design the network architecture using Keras
# downconv + flatconv + downconv + flatconv + flatconv + upconv + flatconv + upconv + flatconv
from tensorflow.python.keras.layers import Input, Conv2D, Conv2DTranspose
from tensorflow.python.keras.models import Model

inputs = Input(shape=(256, 256, 3))
x = Conv2D(32, (3, 3), strides=(2,2), activation='relu', padding='same')(inputs)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(32, (3, 3), strides=(2,2), activation='relu', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (3, 3), strides=(2,2), activation='relu', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (3, 3), strides=(2,2), activation='relu', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
outputs = Conv2D(3, (3, 3), activation='relu', padding='same')(x)
model =Model(inputs, outputs)
model.compile(optimizer=tf.train.AdamOptimizer(0.001), loss='mean_squared_error', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 128, 128, 32)      9248      
__________

In [9]:
import pickle
with open('training_data', 'rb') as file:
    train_images, train_outputs = pickle.load(file)
# Normalize the pixel values
train_images = train_images.astype('float32') / 255
train_outputs = train_outputs.astype('float32') / 255

In [10]:
model.fit(train_images, train_outputs, epochs=5, batch_size=1)

Epoch 1/5
9/9 [==============================] - 6s 661ms/step - loss: 0.1981 - acc: 0.4609
Epoch 2/5
9/9 [==============================] - 4s 488ms/step - loss: 0.0795 - acc: 0.3732
Epoch 3/5
9/9 [==============================] - 4s 482ms/step - loss: 0.0782 - acc: 0.2172
Epoch 4/5
9/9 [==============================] - 4s 481ms/step - loss: 0.0688 - acc: 0.3228
Epoch 5/5
9/9 [==============================] - 4s 496ms/step - loss: 0.0567 - acc: 0.4624
